### Задание 1

В Подсказки по компаниям пробуют вводить названия брендов, которые не всегда совпадают с названием юридического лица.
Например, «Дейта Кью» мы легко подскажем, а вот «Дадата» — уже нет. Хочется это исправить.
Расскажите:
1.	Как бы вы составили справочник брендов (откуда возьмёте информацию, как приведёте к нужному виду, какие видите проблемы или открытые вопросы).
2.	Каким бы он был (структура справочника и пяток записей для примера).

**Описание подхода к решению**

Бренд - это символ (набор символов) какой-либо компании, формирующий ее имидж. Для защиты бренда от использования в чужих интересах все составляющие бренда регистрируются в Роспатенте как торговый знак. Роспатент ведет [открытый реестр товарных знаков и знаков обслуживания РФ](https://rospatent.gov.ru/opendata/7730176088-tz), который обновляется ежемесячно.

Данные представляют из себя csv-файл, где в колонках данные о регистрации и прочих признаках товарного знака, а также ссылка на электронный документ публикации товарного знака ([пример документа](https://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=913526)). В этом документе содержится изображение обозначения товарного знака и прочие сведения (классы классификатора товаров и услуг и проч.) При этом словесное обозначение товарного знака отсутствует как в документе публикации, так и в самом реестре.

Следует отметить, что даже само [заявление для регистрации торгового знака](https://rospatent.gov.ru/ru/stateservices/gosudarstvennaya-registraciya-tovarnogo-znaka-znaka-obsluzhivaniya-kollektivnogo-znaka-i-vydacha-svidetelstv-na-tovarnyy-znak-znak-obsluzhivaniya-kollektivnyy-znak-ih-dublikatov) не содержит отдельного поля для указания словесного обозначения товарного знака. Максимум - указание признака "словесный знак" и текстовое описание заявляемого обозначения, которое заполняется редко. В реестре в колонке вид товарного знака только пустые значения, в документах публикации признак словесного товарного знака указан далеко не по всем позициям, содержащим только слова на изображениях. В изображениях, где кроме слов есть рисунок или особый шрифт, признак словесного знака не указывается, так как элементы дизайна - тоже важная его часть.\
Поэтому все, что мы можем получить из наиболее полного реестра товарных знаков, - это их изображения. 

Однако, с помощью ПО для выделения текста из картинки мы можем получить словесное обозначение бренда из полученных изображений, там, где это возможно. Для этого я использовал наиболее популярное и хорошо себя зарекомендовавшее приложение Tesseract.

Далее на нескольких примерах рассмотрим возможности и ограничения такого подхода.

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

from PIL import Image
from pytesseract import pytesseract
import io

from bs4 import BeautifulSoup
import requests
import re
import time

from IPython.display import Image as show_img
from IPython.core.display import HTML 

Загрузим открытый реестр товарных знаков и знаков обслуживания РФ с сайта Роспатента.

In [2]:
data = pd.read_csv('~//data-20230101-structure-20180828.csv', low_memory=False)\
[['registration number',
  'registration date',
  'right holder name',
  'right holder address',
  'correspondence address',
  'right holder ogrn',
  'right holder inn',
  'kind specification',
  'publication URL']]
data.tail()

,registration number,registration date,right holder name,right holder address,correspondence address,right holder ogrn,right holder inn,kind specification,publication URL
843205,913522,20221230.0,Общество с ограниченной ответственностью «А-ЛАБ»,"123458, Москва, муниципальный округ Строгино в...","125284, Москва, Хорошевское шоссе,24,120, Фили...",1217700011726,7.734440e+09,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...
843206,913523,20221230.0,"Общество с ограниченной ответственностью ""СФЕРА""","249028, Калужская область, Боровский р-н, г. Е...","123290, Москва-290, А/Я 46,а/я 46, Ржевцев Вал...",1204000007958,4.003041e+09,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...
843207,913524,20221230.0,Будко Наталья Анатольевна,"108811, Москва, г. Московский, ул. Москвитина,...","140008, Московская область, г. Люберцы, 3-е По...",315774600107169,7.720709e+11,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...
843208,913525,20221230.0,Германова Юлия Павловна,"156019, Костромская область, г. Кострома, ш. К...","156000, Костромская область, г. Кострома, ул. ...",304440114200095,4.401200e+11,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...
843209,913526,20221230.0,Общество с ограниченной ответственностью «Союз...,"119136, Москва, 3-й Сетуньский пр-д, 10","196601, Санкт-Петербург, Пушкин-1, а/я 61, Теп...",1127746172080,7.729705e+09,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...


In [3]:
data['kind specification'].value_counts(dropna=0)

NaN     843209
\r\n         1
Name: kind specification, dtype: int64

In [4]:
print(f"Датасет содержит {f'{data.shape[0]:,.0f}'.replace(',', ' ')} записей о торговых знаках.")
print(f'''Из них зарегистрировано после 1990 г.: \
{f'{data[data["registration date"] > 19900000].shape[0]:,.0f}'.replace(',', ' ')}.''')
print(f'''После 2000 г.: \
{f'{data[data["registration date"] > 20000000].shape[0]:,.0f}'.replace(',', ' ')}.''')

mean_3_last = round(np.mean([data[data["registration date"] > 20221200].shape[0],
                   data[(data["registration date"] > 20221100) & (data["registration date"] < 20221200)].shape[0],
                   data[(data["registration date"] > 20221000) & (data["registration date"] < 20221100)].shape[0]]))

print(f'''За последний месяц (декабрь 2022): \
{f'{data[data["registration date"] > 20221200].shape[0]:,.0f}'.replace(',', ' ')}.''')
print(f'''В среднем за последние 3 месяца (окт-дек 2022): \
{f'{mean_3_last:,.0f}'.replace(',', ' ')}.''')

Датасет содержит 843 210 записей о торговых знаках.
Из них зарегистрировано после 1990 г.: 826 447.
После 2000 г.: 729 893.
За последний месяц (декабрь 2022): 7 074.
В среднем за последние 3 месяца (окт-дек 2022): 6 173.


Для тестирования ограничимся данными за декабрь.

In [5]:
data_test = data[data["registration date"] > 20221200].reset_index(drop=1)
data_test.tail(3)

,registration number,registration date,right holder name,right holder address,correspondence address,right holder ogrn,right holder inn,kind specification,publication URL
7071,913524,20221230.0,Будко Наталья Анатольевна,"108811, Москва, г. Московский, ул. Москвитина,...","140008, Московская область, г. Люберцы, 3-е По...",315774600107169,7.720709e+11,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...
7072,913525,20221230.0,Германова Юлия Павловна,"156019, Костромская область, г. Кострома, ш. К...","156000, Костромская область, г. Кострома, ул. ...",304440114200095,4.401200e+11,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...
7073,913526,20221230.0,Общество с ограниченной ответственностью «Союз...,"119136, Москва, 3-й Сетуньский пр-д, 10","196601, Санкт-Петербург, Пушкин-1, а/я 61, Теп...",1127746172080,7.729705e+09,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...


Для удобства обрежем названия организационных форм из названий организаций

In [6]:
def get_the_name(x):
    if '«' in x:
        return x.split('«')[1].split('»')[0]
    if '"' in x:
        return x.split('"')[1]
    else: 
        return x

In [7]:
data_test['right holder name cut'] = data_test['right holder name'].apply(lambda x:get_the_name(x))
data_test = data_test.iloc[:,[0,1,2,9,3,4,5,6,7,8]]
data_test.tail(3)

,registration number,registration date,right holder name,right holder name cut,right holder address,correspondence address,right holder ogrn,right holder inn,kind specification,publication URL
7071,913524,20221230.0,Будко Наталья Анатольевна,Будко Наталья Анатольевна,"108811, Москва, г. Московский, ул. Москвитина,...","140008, Московская область, г. Люберцы, 3-е По...",315774600107169,7.720709e+11,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...
7072,913525,20221230.0,Германова Юлия Павловна,Германова Юлия Павловна,"156019, Костромская область, г. Кострома, ш. К...","156000, Костромская область, г. Кострома, ул. ...",304440114200095,4.401200e+11,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...
7073,913526,20221230.0,Общество с ограниченной ответственностью «Союз...,Союз Святого Иоанна Воина,"119136, Москва, 3-й Сетуньский пр-д, 10","196601, Санкт-Петербург, Пушкин-1, а/я 61, Теп...",1127746172080,7.729705e+09,NaN,http://www1.fips.ru/fips_servl/fips_servlet?DB...


Далее создадим функцию для получения ссылки на изображение торгового знака из ссылки на публикацию о торговом знаке.

In [8]:
def get_img_url(url):
    time.sleep(3)
    response=requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'lxml')
    img_url = str(soup.find('img', alt="" ,src=re.compile(".jpg"))).split('"')[1]
    return img_url

Например:

In [9]:
url = data_test['publication URL'][55]
url

'http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=906491'

In [10]:
get_img_url(url)

'http://new.fips.ru/ofpstorage/TM/2022.12.01/RUNWTM/000/000/090/649/100/%D0%A2%D0%97-906491-00001/00000001-m.jpg'

Создадим функцию, извлекающую текстовое обозначение бренда из графического изображения с помощью приложения tesseract. 

Извлечем русский и английский текст

In [11]:
def get_brand_name(img_url):
    web_img = requests.get(img_url)

    path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    pytesseract.tesseract_cmd = path_to_tesseract

    img = Image.open(io.BytesIO(web_img.content))
    text1 = pytesseract.image_to_string(img, lang="rus")
    text2 = pytesseract.image_to_string(img, lang="eng")
    
    return [re.sub('\W|\s',' ',text1).strip(), re.sub('\W|\s',' ',text2).strip()]

Рассмотрим работу функций на нескольких примерах:

In [12]:
examples = data_test.loc[[57,402,4001,4444,7000],'publication URL'].values
examples

array(['http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=906493',
       'http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=906838',
       'http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=910442',
       'http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=910885',
       'http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=913450'],
      dtype=object)

In [13]:
start_time = time.time()
for i in range(len(examples)):
    img_url = get_img_url(examples[i])
    display(show_img(url= img_url))
    print(get_brand_name(img_url))
exec_time = time.time() - start_time
print("\n\n--- %s seconds ---" % round(exec_time,1))

['ЗК', 'wikikid']


['аь', 'Ca']


['О кРУЖЕНИЕ  визнЕс клуБ', 'OKPYKEHUE  suaHEC KnyS']


['ВАУБАВ БАЙСАР', 'BAYSAR BAUCAP']


['Я  22777 И  у ФЫБАКА', 'wy ZZ 7  y PLIBAKA']


--- 24.4 seconds ---


Есть заметные огрехи для торговых знаков с изощренным дизайном, но большинство полученных названий в целом соответствуют изображениям, это радует. Далее можно будет еще потюнить правописание с помощью библиотеки nltk..

Ко всему массиву декабря применить функцию не удалось, заблокировали на сайте Роспатента за бессовестный парсинг)
Благо быстро разблокировали, и я добавил 3-секундный лаг в функцию парсинга, но теперь это просто долго.

In [14]:
print (f'Примерное время выполнения программы для данных за один месяц: \
{round(((exec_time / len(examples)) * mean_3_last) /60 /60,1)} ч')

Примерное время выполнения программы для данных за один месяц: 8.4 ч


Тем не менее с текущей версией алгоритма можно меньше чем за 1 ночь обновить данные по брендам для компаний. И на следующий день пробежаться по массиву вручную. 6-7 тыс строк - это много, но в целом реально. За то данные будут максимально полными.

Финальная таблица будет выглядеть примерно так:

In [15]:
mapping = data_test.loc[[57,402,4001,4444,7000],['right holder name cut','publication URL']]
mapping['logo'] = mapping['publication URL'].apply(lambda x: get_img_url(x))
mapping['brand name'] = mapping['logo'].apply(lambda x: get_brand_name(x))

def path_to_image_html(img_url):
    return '<img src="'+ img_url + '" width="60" >'
mapping.to_html(escape=False, formatters=dict(logo=path_to_image_html))
HTML(mapping.to_html(escape=False,formatters=dict(logo=path_to_image_html)))

,right holder name cut,publication URL,logo,brand name
57,Викикид электронная коммерция,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=906493,,"[ЗК, wikikid]"
402,Тагызаде Эльшан Алисафтар Оглы,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=906838,,"[аь, Ca]"
4001,Маркеттайм,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=910442,,"[О кРУЖЕНИЕ визнЕс клуБ, OKPYKEHUE suaHEC KnyS]"
4444,ПРОМОБОРУДОВАНИЕ,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=910885,,"[ВАУБАВ БАЙСАР, BAYSAR BAUCAP]"
7000,ФИШЕРМАН,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=913450,,"[Я 22777 И у ФЫБАКА, wy ZZ 7 y PLIBAKA]"
